In [53]:
import keras
import random
import requests
import numpy as np
import pandas as pd
from io import StringIO
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate

In [2]:
# get the data
url = 'http://www.kalmasoft.com/KLEX/SAMPLES/KDBGIVE.txt'

r = requests.get(url)
r.encoding='iso8859_6'
df = pd.read_csv(StringIO(r.text[r.text.find('ID\tArabic\tVocalized\tRoman\tCountry\tGender\tVType'):]), delimiter='\t')#, encoding='iso8859_6')
df['Vocalized'] = df['Vocalized'].str.encode('iso8859_6', 'ignore').str.decode('iso8859_6', 'ignore')

In [3]:
df.head()

,ID,Arabic,Vocalized,Roman,Country,Gender,VType
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AGN000001,إبتداء,إبتداء,Ebtida,LB,F,NaN
2,AGN000001-03,ابتداء,ابتداء,Ebtida,LB,F,A
3,AGN000002,إبتداغ,إبتداغ,Ebtida,YE,M,NaN
4,AGN000002-03,ابتداغ,ابتداغ,Ebtida,YE,M,A


In [4]:
df.drop(0, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(2715, 7)

In [5]:
df.head()

,ID,Arabic,Vocalized,Roman,Country,Gender,VType
0,AGN000001,إبتداء,إبتداء,Ebtida,LB,F,NaN
1,AGN000001-03,ابتداء,ابتداء,Ebtida,LB,F,A
2,AGN000002,إبتداغ,إبتداغ,Ebtida,YE,M,NaN
3,AGN000002-03,ابتداغ,ابتداغ,Ebtida,YE,M,A
4,AGN000003,إبتساظ,إبتساظ,Ebtissat,YE,M,F


In [6]:
df[df.ID.apply(lambda id: id[0]=='A')].shape # all names are arabic given names

(2715, 7)

In [57]:
# transform romanized names for prediction
df.Roman = df.Roman.apply(lambda x: '\t' + x + '\n') 

In [58]:
# construct alphabets
arabic_alphabet = ['ا','أ','ب','ت','ث','ج','ح','خ','د','ذ','ر','ز','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ك','ل','م','ن','و','ه','ي','پ','چ','ژ','گ','ک','ی','ۀ','آ','ة','ء','ئ','ؤ','إ']
arabic_alphabet.extend(np.unique(df.Vocalized.apply(lambda x: x[np.argmax([-1 if letter in arabic_alphabet else _ for _,letter in enumerate(x)])])))
for letter in arabic_alphabet:
    if np.all([letter not in x for x in df.Vocalized]):
        arabic_alphabet.remove(letter)
        
assert(df[df.Vocalized.apply(lambda x: np.any([letter not in arabic_alphabet for letter in x]))].empty)

english_alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', 'A', 'E']
english_alphabet.extend([' ', '\t', '\n'])
assert(df[df.Roman.apply(lambda x: np.any([letter not in english_alphabet for letter in x]))].empty)

In [33]:
batch_size = 64  # batch size for training
epochs = 500  # number of epochs to train for
latent_dim = 128  # latent dimensionality of the encoding space

input_characters = sorted(list(arabic_alphabet))
target_characters = sorted(list(english_alphabet))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in df.Vocalized])
max_decoder_seq_length = max([len(txt) for txt in df.Roman])

print('Number of samples:', len(df.Vocalized))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(df.Vocalized), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(df.Vocalized), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(df.Vocalized), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# split into train and test sets
test_indices = random.choices(range(len(encoder_input_data)), k=100)
train_indices = list(set(range(len(encoder_input_data))) - set(test_indices))

df_test = df.loc[test_indices]
df_test.reset_index(drop=True, inplace=True)
df_train = df.loc[train_indices]

Number of samples: 2715
Number of unique input tokens: 39
Number of unique output tokens: 31
Max sequence length for inputs: 11
Max sequence length for outputs: 16


In [42]:
for i, (input_text, target_text) in enumerate(zip(df.Vocalized, df.Roman)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# define an input sequence and process it
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# set up the decoder, using `encoder_states` as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


encoder_input_data_train, encoder_input_data_test = encoder_input_data[train_indices], encoder_input_data[test_indices]
decoder_input_data_train, decoder_input_data_test = decoder_input_data[train_indices], decoder_input_data[test_indices]
decoder_target_data_train, decoder_target_data_test = decoder_target_data[train_indices], decoder_target_data[test_indices]

In [44]:
# define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=keras.optimizers.Adam(lr=0.015), loss='categorical_crossentropy')

In [45]:
# train the model
model.fit([encoder_input_data_train, decoder_input_data_train], decoder_target_data_train,
          batch_size=batch_size,
          epochs=epochs)

Epoch 1/500
2618/2618 [==============================] - 5s 2ms/step - loss: 1.2224
Epoch 2/500
2618/2618 [==============================] - 2s 587us/step - loss: 0.9245
Epoch 3/500
2618/2618 [==============================] - 2s 575us/step - loss: 0.7561
Epoch 4/500
2618/2618 [==============================] - 2s 651us/step - loss: 0.5885
Epoch 5/500
2618/2618 [==============================] - 2s 623us/step - loss: 0.4210
Epoch 6/500
2618/2618 [==============================] - 2s 874us/step - loss: 0.3086
Epoch 7/500
2618/2618 [==============================] - 2s 625us/step - loss: 0.2323
Epoch 8/500
2618/2618 [==============================] - 2s 822us/step - loss: 0.1924
Epoch 9/500
2618/2618 [==============================] - 2s 789us/step - loss: 0.1654
Epoch 10/500
2618/2618 [==============================] - 2s 603us/step - loss: 0.1529
Epoch 11/500
2618/2618 [==============================] - 1s 552us/step - loss: 0.1451
Epoch 12/500
2618/2618 [==============================

2618/2618 [==============================] - 2s 592us/step - loss: 0.1013
Epoch 96/500
2618/2618 [==============================] - 2s 613us/step - loss: 0.1007
Epoch 97/500
2618/2618 [==============================] - 1s 548us/step - loss: 0.1014
Epoch 98/500
2618/2618 [==============================] - 2s 575us/step - loss: 0.1007
Epoch 99/500
2618/2618 [==============================] - 2s 640us/step - loss: 0.1006
Epoch 100/500
2618/2618 [==============================] - 2s 652us/step - loss: 0.1014
Epoch 101/500
2618/2618 [==============================] - 2s 627us/step - loss: 0.0992
Epoch 102/500
2618/2618 [==============================] - 2s 606us/step - loss: 0.0992
Epoch 103/500
2618/2618 [==============================] - 2s 596us/step - loss: 0.0994
Epoch 104/500
2618/2618 [==============================] - 2s 611us/step - loss: 0.0996
Epoch 105/500
2618/2618 [==============================] - 2s 808us/step - loss: 0.0996
Epoch 106/500
2618/2618 [=========================

2618/2618 [==============================] - 2s 722us/step - loss: 0.0979
Epoch 189/500
2618/2618 [==============================] - 2s 607us/step - loss: 0.0983
Epoch 190/500
2618/2618 [==============================] - 1s 567us/step - loss: 0.0980
Epoch 191/500
2618/2618 [==============================] - 2s 643us/step - loss: 0.0981
Epoch 192/500
2618/2618 [==============================] - 2s 629us/step - loss: 0.0979
Epoch 193/500
2618/2618 [==============================] - 2s 622us/step - loss: 0.0981
Epoch 194/500
2618/2618 [==============================] - 1s 552us/step - loss: 0.0989
Epoch 195/500
2618/2618 [==============================] - 2s 658us/step - loss: 0.0977
Epoch 196/500
2618/2618 [==============================] - 2s 663us/step - loss: 0.0987
Epoch 197/500
2618/2618 [==============================] - 1s 564us/step - loss: 0.0971
Epoch 198/500
2618/2618 [==============================] - 1s 564us/step - loss: 0.0977
Epoch 199/500
2618/2618 [=====================

2618/2618 [==============================] - 2s 722us/step - loss: 0.0969
Epoch 282/500
2618/2618 [==============================] - 2s 781us/step - loss: 0.0969
Epoch 283/500
2618/2618 [==============================] - 2s 647us/step - loss: 0.0972
Epoch 284/500
2618/2618 [==============================] - 2s 782us/step - loss: 0.0970
Epoch 285/500
2618/2618 [==============================] - 2s 874us/step - loss: 0.0968
Epoch 286/500
2618/2618 [==============================] - 2s 610us/step - loss: 0.0964
Epoch 287/500
2618/2618 [==============================] - 2s 591us/step - loss: 0.0968
Epoch 288/500
2618/2618 [==============================] - 2s 649us/step - loss: 0.0979
Epoch 289/500
2618/2618 [==============================] - 2s 634us/step - loss: 0.0979
Epoch 290/500
2618/2618 [==============================] - 2s 621us/step - loss: 0.1004
Epoch 291/500
2618/2618 [==============================] - 2s 667us/step - loss: 0.1023
Epoch 292/500
2618/2618 [=====================

2618/2618 [==============================] - 1s 483us/step - loss: 0.0961 0s - lo
Epoch 375/500
2618/2618 [==============================] - 1s 464us/step - loss: 0.0958
Epoch 376/500
2618/2618 [==============================] - 1s 454us/step - loss: 0.0955
Epoch 377/500
2618/2618 [==============================] - 1s 444us/step - loss: 0.0958
Epoch 378/500
2618/2618 [==============================] - 1s 464us/step - loss: 0.0961
Epoch 379/500
2618/2618 [==============================] - 1s 494us/step - loss: 0.0960
Epoch 380/500
2618/2618 [==============================] - 1s 448us/step - loss: 0.0965
Epoch 381/500
2618/2618 [==============================] - 1s 443us/step - loss: 0.0967
Epoch 382/500
2618/2618 [==============================] - 1s 446us/step - loss: 0.0957
Epoch 383/500
2618/2618 [==============================] - 1s 520us/step - loss: 0.0957
Epoch 384/500
2618/2618 [==============================] - 1s 408us/step - loss: 0.0961
Epoch 385/500
2618/2618 [=============

2618/2618 [==============================] - 1s 403us/step - loss: 0.0963
Epoch 468/500
2618/2618 [==============================] - 1s 508us/step - loss: 0.0960
Epoch 469/500
2618/2618 [==============================] - 1s 493us/step - loss: 0.0965
Epoch 470/500
2618/2618 [==============================] - 1s 469us/step - loss: 0.0975
Epoch 471/500
2618/2618 [==============================] - 1s 469us/step - loss: 0.0977
Epoch 472/500
2618/2618 [==============================] - 1s 507us/step - loss: 0.0976
Epoch 473/500
2618/2618 [==============================] - 1s 504us/step - loss: 0.0987
Epoch 474/500
2618/2618 [==============================] - 1s 487us/step - loss: 0.0996
Epoch 475/500
2618/2618 [==============================] - 1s 546us/step - loss: 0.1039
Epoch 476/500
2618/2618 [==============================] - 1s 540us/step - loss: 0.1246
Epoch 477/500
2618/2618 [==============================] - 1s 490us/step - loss: 0.1230
Epoch 478/500
2618/2618 [=====================

In [59]:
# define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    # generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # the first character of target sequence is the start character
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # generate a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
    return decoded_sentence

In [51]:
for seq_index in range(0,100,3):
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', df_test.Vocalized[seq_index])
    print('Decoded sentence:', decoded_sentence[:-1])
    print('Target sentence:', df_test.Roman[seq_index][1:])

-
Input sentence: افجزنفٌ
Decoded sentence: Eldjazoli
Target sentence: Eldjazoli

-
Input sentence: اغراب
Decoded sentence: Arab
Target sentence: Arab

-
Input sentence: أجدف
Decoded sentence: Adjdel
Target sentence: Ajdal

-
Input sentence: افدارٍ
Decoded sentence: Aldaghiriy
Target sentence: Eddaghiri

-
Input sentence: افبلسج
Decoded sentence: Albanafsaj
Target sentence: Elbanafsadj

-
Input sentence: افرحٍفٍ
Decoded sentence: Arrahiyliy
Target sentence: Elrohaili

-
Input sentence: افجبٍرٍ
Decoded sentence: Aljubayriy
Target sentence: Eldjobairi

-
Input sentence: افرصاص
Decoded sentence: Errosas
Target sentence: Arrasas

-
Input sentence: افدمشال
Decoded sentence: Eddahchane
Target sentence: Eddahchane

-
Input sentence: افزكزكٍ
Decoded sentence: Azzamzamiy
Target sentence: Ezzamzami

-
Input sentence: افحجازٌ
Decoded sentence: Elhidjazi
Target sentence: Alhijaziy

-
Input sentence: أشظر
Decoded sentence: Achter
Target sentence: Ashtar

-
Input sentence: افرباغٍ
Decoded sentence: 